In [ ]:
pip install psycopg2

In [4]:
import psycopg2
import csv

In [10]:
# Connect to the ElephantSQL database
conn = psycopg2.connect(
    host="drona.db.elephantsql.com",
    port="5432",
    database="jgnwunhy",
    user="jgnwunhy",
    password="cRXFIUriYT3-N4S6gTCNKNWwzsLU2qFq"
)

# Open a cursor to perform database operations
cur = conn.cursor()

# Create tables
cur.execute('''
    CREATE TABLE IF NOT EXISTS actor (
        actor_id SERIAL PRIMARY KEY,
        actor VARCHAR(200) NOT NULL
    )
''')

cur.execute('''
    CREATE TABLE IF NOT EXISTS award (
        award_id SERIAL PRIMARY KEY,
        award VARCHAR(100) NOT NULL
    )
''')

cur.execute('''
    CREATE TABLE IF NOT EXISTS category (
        category_id SERIAL PRIMARY KEY,
        category VARCHAR(100) NOT NULL
    )
''')

cur.execute('''
    CREATE TABLE IF NOT EXISTS title (
        title_id SERIAL PRIMARY KEY,
        title VARCHAR(200) NOT NULL
    )
''')

cur.execute('''
    CREATE TABLE IF NOT EXISTS network (
        network_id SERIAL PRIMARY KEY,
        network VARCHAR(20) NOT NULL
    )
''')

cur.execute('''
    CREATE TABLE IF NOT EXISTS nomination (
        nomination_id SERIAL PRIMARY KEY,
        award_id INT NOT NULL,
        year INT NOT NULL,
        title_id INT NULL,
        actor_id INT NULL,
        network_id INT NOT NULL,
        category_id INT NOT NULL,
        role VARCHAR(100) NULL,
        episode VARCHAR(200) NULL,
        producers VARCHAR(1000) NULL,
        winner BOOLEAN NOT NULL,
        FOREIGN KEY (award_id) REFERENCES award (award_id),
        FOREIGN KEY (title_id) REFERENCES title (title_id),
        FOREIGN KEY (actor_id) REFERENCES actor (actor_id),
        FOREIGN KEY (network_id) REFERENCES network (network_id),
        FOREIGN KEY (category_id) REFERENCES category (category_id)
    )
''')

cur.execute('''
    CREATE TABLE IF NOT EXISTS title_category (
        title_id INT NOT NULL,
        category_id INT NOT NULL,
        PRIMARY KEY (title_id, category_id),
        FOREIGN KEY (title_id) REFERENCES title (title_id),
        FOREIGN KEY (category_id) REFERENCES category (category_id)
    )
''')

cur.execute('''
    CREATE TABLE IF NOT EXISTS imdb_top_250 (
        imdb_title_id SERIAL PRIMARY KEY,
        title_id INT NOT NULL,
        rank INT NULL,
        years_running VARCHAR(200) NULL,
        certificate VARCHAR(20) NULL,
        rating FLOAT NULL,
        votes INT NULL,
        synopsis VARCHAR(1000) NOT NULL,
        FOREIGN KEY (title_id) REFERENCES title (title_id)
    )
''')

# Commit the table creations
conn.commit()

# Load data from CSV files
def load_data(table, csv_file):
    with open(csv_file, 'r', newline='', encoding='utf-8') as file:
        reader = csv.reader(file)
        next(reader)  # Skip the header row
        for row in reader:
            insert_query = f"INSERT INTO {table} VALUES ({','.join(['%s'] * len(row))})"
            cur.execute(insert_query, row)

# Define a dictionary mapping table names to CSV file paths
table_csv_mapping = {
    "title": "C:\\Users\\8saks\\Documents\\GitHub\\ETL_IMDB\\Sakshi\\titles.csv",
    "actor": "C:\\Users\\8saks\\Documents\\GitHub\\ETL_IMDB\\Sakshi\\actors.csv",
    "award": "C:\\Users\\8saks\\Documents\\GitHub\\ETL_IMDB\\Sakshi\\awards.csv",
    "category": "C:\\Users\\8saks\\Documents\\GitHub\\ETL_IMDB\\Sakshi\\categories.csv",
    "network": "C:\\Users\\8saks\\Documents\\GitHub\\ETL_IMDB\\Sakshi\\network.csv",
    "nomination": "C:\\Users\\8saks\\Documents\\GitHub\\ETL_IMDB\\Sakshi\\nominations.csv",
    "title_category": "C:\\Users\\8saks\\Documents\\GitHub\\ETL_IMDB\\Sakshi\\title_categories.csv",
    "imdb_top_250": "C:\\Users\\8saks\\Documents\\GitHub\\ETL_IMDB\\Sakshi\\imdb_top_250.csv"
}

# Load data into tables from CSV files
for table, csv_file in table_csv_mapping.items():
    load_data(table, csv_file)

# Commit the changes and close the cursor and connection
conn.commit()
cur.close()
conn.close()
